In [1]:
import os
from google.cloud import language_v1
import pandas as pd
from cleantext import clean
import json
import glob
from pprint import pprint
import datetime
import re  
import nltk  
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
nltk.download()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ashlyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
bitcoin_path = "Resources/bitcoin.csv"

bitcoin = pd.read_csv(bitcoin_path)

bitcoin.head(50)

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4,5,6,7,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False
5,ZerrBenz™ ⚔ ✪ 20732,"Bkk, Thailand",I'm a cat slave 🐱 Interested in Blockchain · T...,2010-01-12 07:00:04,742.0,716,2444,False,2021-02-10 23:53:30,💹 Trade #Crypto on #Binance \n\n📌 Enjoy #Cashb...,"['Crypto', 'Binance', 'Cashback']",Twitter Web App,False
6,Bitcoin-Bot,"Florida, USA",Bot to generate Bitcoin picture as combination...,2019-12-23 16:49:16,131.0,84,5728,False,2021-02-10 23:53:17,&lt;'fire' &amp; 'man'&gt;\n#Bitcoin #Crypto #...,"['Bitcoin', 'Crypto', 'BTC']",BTC_p_bot,False
7,Cryptocurrencies / EUR,NaN,Stay updated on the main cryptocurrencies of t...,2018-08-03 21:30:08,4052.0,1,9,False,2021-02-10 23:52:42,🔄 Prices update in $EUR (1 hour):\n\n$BTC - ...,NaN,Cryptocurrencies prices to EUR,False
8,Mikcoin,NaN,"Technical Analyst | Trader\n\nNo certainty, on...",2020-11-26 23:45:46,104.0,41,238,False,2021-02-10 23:52:25,#BTC #Bitcoin #Ethereum #ETH #Crypto #cryptotr...,"['BTC', 'Bitcoin', 'Ethereum', 'ETH', 'Crypto'...",Twitter Web App,False
9,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:52:08,.@Tesla’s #bitcoin investment is revolutionary...,"['bitcoin', 'crypto']",Twitter Web App,False


In [3]:
df = bitcoin.filter(['text','date'])[0:100000]

df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))


,text,date
0,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04
1,"Today, that's this #Thursday, we will do a "" ...",2021-02-10 23:58:48
2,"Guys evening, I have read this article about B...",2021-02-10 23:54:48
3,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33
4,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06
...,...,...
99995,@NftCollector31 A #legendary #NFT item from \n...,2021-04-19 10:15:00
99996,@NftCryptosArt A #legendary #NFT item from \n@...,2021-04-19 10:14:43
99997,@SnowGotMelted @NFTE @NFTCollector11 @KennethB...,2021-04-19 10:14:28
99998,@NftPasso A #legendary #NFT item from \n@neond...,2021-04-19 10:14:19


In [4]:
def tweet_to_words(tweet):
    text = tweet.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    words = text.split()
    words = [w for w in words if w not in stopwords.words("english")]
    words = [PorterStemmer().stem(w) for w in words]
    return words

In [5]:
cleantext=[]
for item in tqdm(df['text']):
    words=tweet_to_words(item)
    cleantext+=[words]
df['cleaned']=cleantext
df

100%|██████████████████████████████████| 100000/100000 [05:25<00:00, 307.60it/s]


,text,date,cleaned
0,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04,"[blue, ridg, bank, share, halt, nyse, bitcoin,..."
1,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10 23:58:48,"[today, thursday, take, 2, friend, leowandersl..."
2,"Guys evening, I have read this article about B...",2021-02-10 23:54:48,"[guy, even, read, articl, btc, would, like, sh..."
3,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33,"[btc, big, chanc, billion, price, 4872644, 0, ..."
4,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06,"[network, secur, 9, 508, node, today, soon, bi..."
...,...,...,...
99995,@NftCollector31 A #legendary #NFT item from \n...,2021-04-19 10:15:00,"[nftcollector31, legendari, nft, item, neondis..."
99996,@NftCryptosArt A #legendary #NFT item from \n@...,2021-04-19 10:14:43,"[nftcryptosart, legendari, nft, item, neondist..."
99997,@SnowGotMelted @NFTE @NFTCollector11 @KennethB...,2021-04-19 10:14:28,"[snowgotmelt, nfte, nftcollector11, kennethbos..."
99998,@NftPasso A #legendary #NFT item from \n@neond...,2021-04-19 10:14:19,"[nftpasso, legendari, nft, item, neondistrictr..."


In [6]:
def unlist(list):
    words=''
    for item in list:
        words+=item+' '
    return words

In [7]:
def compute_vader_scores(df, label):
    sid = SentimentIntensityAnalyzer()
    df["negative"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neg"])
    df["neutral"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["neu"])
    df["positive"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["pos"])
    df["comp"] = df[label].apply(lambda x: sid.polarity_scores(unlist(x))["compound"])
    return df

In [8]:
final_bitcoin = compute_vader_scores(df,'cleaned')
final_bitcoin

,text,date,cleaned,negative,neutral,positive,comp
0,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04,"[blue, ridg, bank, share, halt, nyse, bitcoin,...",0.0,0.872,0.128,0.2960
1,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10 23:58:48,"[today, thursday, take, 2, friend, leowandersl...",0.0,0.775,0.225,0.4939
2,"Guys evening, I have read this article about B...",2021-02-10 23:54:48,"[guy, even, read, articl, btc, would, like, sh...",0.0,0.719,0.281,0.5719
3,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33,"[btc, big, chanc, billion, price, 4872644, 0, ...",0.0,1.000,0.000,0.0000
4,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06,"[network, secur, 9, 508, node, today, soon, bi...",0.2,0.800,0.000,-0.5423
...,...,...,...,...,...,...,...
99995,@NftCollector31 A #legendary #NFT item from \n...,2021-04-19 10:15:00,"[nftcollector31, legendari, nft, item, neondis...",0.0,1.000,0.000,0.0000
99996,@NftCryptosArt A #legendary #NFT item from \n@...,2021-04-19 10:14:43,"[nftcryptosart, legendari, nft, item, neondist...",0.0,1.000,0.000,0.0000
99997,@SnowGotMelted @NFTE @NFTCollector11 @KennethB...,2021-04-19 10:14:28,"[snowgotmelt, nfte, nftcollector11, kennethbos...",0.0,1.000,0.000,0.0000
99998,@NftPasso A #legendary #NFT item from \n@neond...,2021-04-19 10:14:19,"[nftpasso, legendari, nft, item, neondistrictr...",0.0,1.000,0.000,0.0000


In [ ]:
# print(len(date), len(final_bitcoin.cleaned),len(final_bitcoin.negative),len(final_bitcoin.comp),len(final_bitcoin.neutral))

In [10]:
# final_bitcoin.isnull().values.any()

final_bitcoin[['dates','Time','empty']] = final_bitcoin.date.str.split(" ",expand=True)

final_bitcoin

,text,date,cleaned,negative,neutral,positive,comp,dates,Time,empty
0,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04,"[blue, ridg, bank, share, halt, nyse, bitcoin,...",0.0,0.872,0.128,0.2960,2021-02-10,23:59:04,None
1,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10 23:58:48,"[today, thursday, take, 2, friend, leowandersl...",0.0,0.775,0.225,0.4939,2021-02-10,23:58:48,None
2,"Guys evening, I have read this article about B...",2021-02-10 23:54:48,"[guy, even, read, articl, btc, would, like, sh...",0.0,0.719,0.281,0.5719,2021-02-10,23:54:48,None
3,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33,"[btc, big, chanc, billion, price, 4872644, 0, ...",0.0,1.000,0.000,0.0000,2021-02-10,23:54:33,None
4,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06,"[network, secur, 9, 508, node, today, soon, bi...",0.2,0.800,0.000,-0.5423,2021-02-10,23:54:06,None
...,...,...,...,...,...,...,...,...,...,...
99995,@NftCollector31 A #legendary #NFT item from \n...,2021-04-19 10:15:00,"[nftcollector31, legendari, nft, item, neondis...",0.0,1.000,0.000,0.0000,2021-04-19,10:15:00,None
99996,@NftCryptosArt A #legendary #NFT item from \n@...,2021-04-19 10:14:43,"[nftcryptosart, legendari, nft, item, neondist...",0.0,1.000,0.000,0.0000,2021-04-19,10:14:43,None
99997,@SnowGotMelted @NFTE @NFTCollector11 @KennethB...,2021-04-19 10:14:28,"[snowgotmelt, nfte, nftcollector11, kennethbos...",0.0,1.000,0.000,0.0000,2021-04-19,10:14:28,None
99998,@NftPasso A #legendary #NFT item from \n@neond...,2021-04-19 10:14:19,"[nftpasso, legendari, nft, item, neondistrictr...",0.0,1.000,0.000,0.0000,2021-04-19,10:14:19,None


In [11]:
final_bitcoin = final_bitcoin.filter(['dates', 'cleaned','negative','neutral','positive','comp'])
final_bitcoin.head(40)

,dates,cleaned,negative,neutral,positive,comp
0,2021-02-10,"[blue, ridg, bank, share, halt, nyse, bitcoin,...",0.000,0.872,0.128,0.2960
1,2021-02-10,"[today, thursday, take, 2, friend, leowandersl...",0.000,0.775,0.225,0.4939
2,2021-02-10,"[guy, even, read, articl, btc, would, like, sh...",0.000,0.719,0.281,0.5719
3,2021-02-10,"[btc, big, chanc, billion, price, 4872644, 0, ...",0.000,1.000,0.000,0.0000
4,2021-02-10,"[network, secur, 9, 508, node, today, soon, bi...",0.200,0.800,0.000,-0.5423
5,2021-02-10,"[trade, crypto, binanc, enjoy, cashback, 10, t...",0.000,0.824,0.176,0.4939
6,2021-02-10,"[lt, fire, amp, man, gt, bitcoin, crypto, btc,...",0.178,0.667,0.156,-0.0772
7,2021-02-10,"[price, updat, eur, 1, hour, btc, 37082, 1, 0,...",0.000,1.000,0.000,0.0000
8,2021-02-10,"[btc, bitcoin, ethereum, eth, crypto, cryptotr...",0.000,1.000,0.000,0.0000
9,2021-02-10,"[tesla, bitcoin, invest, revolutionari, crypto...",0.000,1.000,0.000,0.0000


In [12]:
final_bitcoin.to_csv('bitcoin.csv')
